<a href="https://colab.research.google.com/github/Amanpatni211/Cat-or-No-Cat/blob/main/Sentence_classification_iitpatna_komal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/ncg-task/training-data.git "/content/train"

Cloning into '/content/train'...
remote: Enumerating objects: 6864, done.
remote: Counting objects: 100% (3083/3083), done.
remote: Compressing objects: 100% (2728/2728), done.
remote: Total 6864 (delta 567), reused 2504 (delta 279), pack-reused 3781
Receiving objects: 100% (6864/6864), 157.36 MiB | 32.19 MiB/s, done.
Resolving deltas: 100% (660/660), done.
Checking out files: 100% (3286/3286), done.


In [3]:
#### Loading Training Data and store it
input_dir = "/content/train/"
import os
import torch
import numpy as np
import time
from torch.utils.data import Dataset, DataLoader
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
list_of_folders = ["query_wellformedness", "passage_re-ranking", "part-of-speech_tagging", 
         "sentence_compression", "sentiment_analysis", "temporal_information_extraction", 
         "phrase_grounding", "text_generation", "text-to-speech_synthesis", 
         "smile_recognition", "topic_models", "question_generation", 
         "relation_extraction", "paraphrase_generation", "question_similarity", 
         "question_answering", "sentence_classification", "prosody_prediction", 
         "semantic_role_labeling", "text_summarization", "semantic_parsing",
         "sarcasm_detection", "natural_language_inference", "negation_scope_resolution"]
input_stanza_list = []
input_stanza_len = []
input_sent_num_list = []
file_name_list = []
for fls in list_of_folders:
  count=0
  for i in os.listdir(input_dir + fls + '/'):
    count=count+1
    for files in os.listdir(input_dir + fls + '/' + str(i)):
      if files.endswith("Stanza-out.txt"):
        stanza_file = open(input_dir + fls + '/' + str(i) + '/' + files, "r")
        stanza_lines = stanza_file.read()
        stanza_lines_list = list(filter(None,map(lambda x:x.lower(),stanza_lines.splitlines()))) # filter empty strings and split into lines
        input_stanza_len.append(len(stanza_lines_list))
        input_stanza_list.append(stanza_lines_list)
      if files.endswith("sentences.txt"):
        sentence_file = open(input_dir + fls + '/' + str(i) + '/' + files, "r")
        sentence_num_list = list(filter(None,sentence_file.read().splitlines())) # filter empty strings and split into lines
        input_sent_num_list.append(sentence_num_list)
    file_name_list.append(fls + '/' + str(i))
  print("Training complete",fls, count)

Training complete query_wellformedness 1
Training complete passage_re-ranking 2
Training complete part-of-speech_tagging 8
Training complete sentence_compression 4
Training complete sentiment_analysis 52
Training complete temporal_information_extraction 2
Training complete phrase_grounding 1
Training complete text_generation 6
Training complete text-to-speech_synthesis 3
Training complete smile_recognition 1
Training complete topic_models 1
Training complete question_generation 2
Training complete relation_extraction 14
Training complete paraphrase_generation 2
Training complete question_similarity 1
Training complete question_answering 6
Training complete sentence_classification 3
Training complete prosody_prediction 1
Training complete semantic_role_labeling 5
Training complete text_summarization 15
Training complete semantic_parsing 3
Training complete sarcasm_detection 2
Training complete natural_language_inference 101
Training complete negation_scope_resolution 1


In [4]:
!git clone https://github.com/ncg-task/trial-data.git "/content/valid"

Cloning into '/content/valid'...
remote: Enumerating objects: 1636, done.
remote: Counting objects: 100% (1636/1636), done.
remote: Compressing objects: 100% (1438/1438), done.
remote: Total 1636 (delta 580), reused 1224 (delta 170), pack-reused 0
Receiving objects: 100% (1636/1636), 27.88 MiB | 22.13 MiB/s, done.
Resolving deltas: 100% (580/580), done.


In [5]:
##### Loading valid dataset
val_input_dir = "/content/valid/"
val_list_of_folders = ["machine-translation", "named-entity-recognition", "question-answering",
         "relation-classification", "text-classification"]
val_input_stanza_list = []
val_input_sent_num_list = []
val_file_name_list = []
val_input_stanza_len = []
for fls in val_list_of_folders:
  count=0
  for i in os.listdir(val_input_dir + fls + '/'):
    count=count+1
    for files in os.listdir(val_input_dir + fls + '/' + str(i)):
      if files.endswith("Stanza-out.txt"):
        stanza_file = open(val_input_dir + fls + '/' + str(i) + '/' + files, "r")
        stanza_lines = stanza_file.read()
        stanza_lines_list = list(filter(None, map(lambda x:x.lower(),stanza_lines.splitlines()))) # filter empty strings and split into lines
        val_input_stanza_len.append(len(stanza_lines_list))
        val_input_stanza_list.append(stanza_lines_list)
      if files.endswith("sentences.txt"):
        sentence_file = open(val_input_dir + fls + '/' + str(i) + '/' + files, "r")
        sentence_num_list = list(filter(None,sentence_file.read().splitlines())) # filter empty strings and split into lines
        val_input_sent_num_list.append(sentence_num_list)
    val_file_name_list.append(fls + '/' + str(i))
  print("Valid complete",fls,count)

Valid complete machine-translation 10
Valid complete named-entity-recognition 10
Valid complete question-answering 10
Valid complete relation-classification 10
Valid complete text-classification 10


In [6]:
#### string to int 
#### conversion for Train set
input_sent_num_list = [[int(s) for s in sublist] for sublist in input_sent_num_list] # convert sentence list string to integer
input_sent_num_list = [list(set(x)) for x in input_sent_num_list]

#### conversion for Valid set
val_input_sent_num_list = [[int(s) for s in sublist] for sublist in val_input_sent_num_list] # convert sentence list string to integer
val_input_sent_num_list = [list(set(x)) for x in val_input_sent_num_list]

In [7]:
#### Ground truth label formation for classification
#### Train set
multihot_input_sent = []
for i in range(len(input_stanza_list)):
  temp =[0]*input_stanza_len[i]
  for j in range(len(input_sent_num_list[i])):
    t1 = input_sent_num_list[i][j] -1
    temp[t1] = 1
  multihot_input_sent.append(temp)
#### valid set
val_multihot_input_sent = []
for i in range(len(val_input_stanza_list)):
  temp =[0]*val_input_stanza_len[i]
  for j in range(len(val_input_sent_num_list[i])):
    t1 = val_input_sent_num_list[i][j] -1
    temp[t1] = 1
  val_multihot_input_sent.append(temp)

In [8]:
## Function for flattening 2d list to 1d list
from collections import Iterable
def flatten(lis):
  for item in lis:
    if isinstance(item, Iterable) and not isinstance(item, str):
      for x in flatten(item):
        yield x
    else:        
      yield item

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  


In [9]:
#### Flatten the list and use heuristic to remove vague sentences
#### Train set
from collections import Counter

train_sentences = list(flatten(input_stanza_list))
train_label = list(flatten(multihot_input_sent))

train_tuple = list(set((zip(train_sentences,train_label))))
train_in_sentences = []
train_sent_label = []
for stan,lab in train_tuple:
  if len(stan) >4:
    train_in_sentences.append(stan)
    train_sent_label.append(lab)

valid_sentences = list(flatten(val_input_stanza_list))
valid_label = list(flatten(val_multihot_input_sent))

valid_tuple = list(set((zip(valid_sentences,valid_label))))
valid_in_sentences = []
valid_sent_label = []
for stan,lab in valid_tuple:
  if len(stan) >4:
    valid_in_sentences.append(stan)
    valid_sent_label.append(lab)

print("train size",len(train_in_sentences),len(train_sent_label))
print("valid size",len(valid_in_sentences),len(valid_sent_label))

train size 51296 51296
valid size 10681 10681


In [10]:
#### Five timesOversampling of minority class in train set
train_tuple = list(set((zip(train_in_sentences,train_sent_label))))
train_sum_sent = 5 * [stan for stan,label in train_tuple if label==1]
train_nonsum_sent = [stan for stan,label in train_tuple if label==0]
train_in_sentences = train_sum_sent + train_nonsum_sent
train_sent_label = len(train_sum_sent)*[1] + len(train_nonsum_sent)*[0]
print("Oversampled train size",len(train_in_sentences),len(train_sent_label))

Oversampled train size 71396 71396


MODEL A tensorflow simple dense model


In [14]:
import json
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [15]:
training_sentences = train_in_sentences
training_labels = train_sent_label
testing_sentences = valid_in_sentences
testing_labels = valid_sent_label

In [16]:
len(training_sentences)

71396

In [19]:
embedding_dim = 16
max_length = 10000000
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 71396

In [ ]:
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [ ]:
# Need this block to get it to work with TensorFlow 2.x
import numpy as np
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)